# Import packages

In [26]:
import numpy as np
import time
from copy import deepcopy

# PQR

In [27]:
def PQR(matrix, n, m, r = 0, eps = 0):

    if not r or r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))
    P = np.arange(m)

    tol = eps + 1
    i = 0

    while i < r and tol > eps:

        norms = np.array([np.linalg.norm(matrix[:, j]) for j in range(i, m)])
        tol = np.sqrt(np.sum(norms**2))
        args = np.argsort(norms)[::-1]
        args += i

        P[[i, args[0]]] = P[[args[0], i]]
        R[:, [i, args[0]]]  = R[:, [args[0], i]]
        matrix[:, [i, args[0]]] = matrix[:, [args[0], i]]

        R[i, i] = norms[args[0] - i]
        Q[:, i] = matrix[:, i] / R[i, i]

        for j in range(i + 1, m):
            R[i, j] = matrix[:, j] @ Q[:, i]
            matrix[:, j] -= R[i, j] * Q[:, i]

        i += 1

    return Q, R, P, i

# MaxVol

In [28]:
def maxvol(mat, n, m, r, eps = 0):

    if not r or r > min(n, m):
          r = min(n, m)

    rows = np.random.permutation(n)
    B = mat[rows[:r], :]
    _, _, cols, rank = PQR(B, r, m, r, eps)


    rev_rows = np.zeros(n, dtype=int)
    rev_cols = np.zeros(m, dtype=int)

    for i in range(n):
        rev_rows[rows[i]] = i
    for i in range(m):
        rev_cols[cols[i]] = i

    matrix = mat[np.ix_(rows, cols)]

    R = np.arange(r)

    C = matrix[:, :r]
    A = C[:r, :]

    inv_A = np.linalg.inv(A)

    prod = C @ inv_A

    max_val_ind = np.unravel_index(np.argmax(np.abs(prod), axis=None), prod.shape)

    iter = 0

    while np.abs(prod[max_val_ind]) > 1 and max_val_ind[0] >= r and iter < r * r:

        iter += 1

        vec = prod[max_val_ind[0], :].copy()
        vec[max_val_ind[1]] -= 1
        vec /= prod[max_val_ind]

        prod -= prod[:, max_val_ind[1]].reshape((-1, 1)) @ vec.reshape((1, -1))

        R[max_val_ind[1]] = max_val_ind[0]

        max_val_ind = np.unravel_index(np.argmax(np.abs(prod), axis=None), prod.shape)

    prod = prod[rev_rows, :]
    R = matrix[np.ix_(R, rev_cols)]

    return prod, R

# Cross Approximation

In [29]:
def cross_approx(matrix, n, m, r = 0, eps = 0):

    if not r and r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))

    vec = np.random.permutation(m)

    i = 0

    max_el = eps + 1

    while i < r and max_el * np.sqrt((n - i) * (m - i)) > eps:

        col = vec[i]

        row = np.argmax(np.abs(matrix[:, col]), axis=None)
        col = np.argmax(np.abs(matrix[row, :]), axis=None)
        row = np.argmax(np.abs(matrix[:, col]), axis=None)

        Q[:, i] = matrix[:, col] / matrix[row, col]
        R[i, :] = matrix[row, :]

        max_el = np.abs(matrix[row, col])

        matrix -= Q[:, i].reshape((-1, 1)) @ R[i, :].reshape((1, -1))

        i += 1

    return Q, R, i

# Create matrix

In [45]:
n = 5000
m = 10000
mat_cr = np.zeros((n, m))

for i in range(n):
    for j in range(m):
        mat_cr[i, j] = np.cos(i + j)

In [52]:
matrix = mat_cr.copy()
mat_copy = mat_cr.copy()

# Run Cross Approximation

In [49]:
start_time = time.time()
Q, R, rank = cross_approx(matrix, n, m, 2)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy) / np.linalg.norm(mat_copy))

time = 0.5917322635650635
1.1744337776112075e-16


# Run PQR

In [51]:
start_time = time.time()
Q, R, P, rank = PQR(matrix, n, m, 2)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy[:, P]) / np.linalg.norm(mat_copy))

time = 1.057051420211792
7.227098604733357e-16


# Run MaxVol

In [53]:
start_time = time.time()
Q, R = maxvol(matrix, n, m, 2)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy) / np.linalg.norm(mat_copy))

time = 0.6902766227722168
1.296308361851803e-16
